# Using Jurassic-2 Grande Instruct on SageMaker through Model Packages

This sample notebook shows you how to deploy **Jurassic-2 Grande Instruct** using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Interact with the model](#B.-Interact-with-the-model)
   3. [Give it an instruction](#C.-Give-it-an-instruction)
   4. [Create your own product description!](#D.-Create-your-own-product-description!)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [1]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/j2-grande-instruct-v1-0-24-4405c0fcbe4e3bf3a467b8091db6dc3c"
}

In [3]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [4]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [5]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [6]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [7]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [8]:
model_name = "j2-grande-instruct"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.12xlarge"
)

### A. Create an endpoint

In [10]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

---------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-2 Grande Instruct as a smart auto-completion algorithm: give it some text as input and it will generate relevant text to naturally complete your input.

These two helpful concepts are worth being familiar with:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generates.

Enter a simple prompt: "To be, or", and let the model complete it

In [11]:
response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt="To be, or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

 not to be: that is the question


As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Give it an instruction

This model was specifically trained to follow natural language instructions. It is the most natural way to interact with large language models: simply tell the model what you want it to do, and it will follow.

For this notebook, we will apply the model to creating a product description for an eCommerce site. Simply tell the model what you want it to do with a simple instruction.

In [12]:
instruction = """Write an engaging product description for clothing eCommerce site.
Product: Humor Men's Graphic T-Shirt.
Description:

"""

response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

This funny t-shirt is perfect for anyone who enjoys a good laugh. It's made of 100% cotton and is soft and comfortable. The graphic on the front is bright and colorful, and it's sure to get attention. Whether you're wearing it to the gym, out with friends, or just around the house, this t-shirt is sure to make you smile.


#### Adjust the parameters
A useful parameter is the temperature. **You can increase creativity by tweaking the temperature.** With temperature 0, the model will always choose the most probable completion, so it will always be the same. Increasing the temperature will provide varying completions, where the completion may be different with every generation. We recommend starting with a value of 0.7.

In [14]:
instruction = """Write an engaging product description for clothing eCommerce site.
Product: Humor Men's Graphic T-Shirt.
Description:

"""

response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

# Note that multiple completions - this is due to numResults > 1
for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

You've been looking for the perfect shirt to wear to your next party, but haven't found anything quite right. This funny graphic t-shirt is perfect for any occasion! It's made of soft, comfortable fabric and features a hilarious graphic that will make you smile. Whether you're going out with friends or staying in for the night, this shirt is sure to turn heads. So don't wait any longer, order your shirt today!
Are you looking for a funny t-shirt that makes people laugh? If so, look no further than this humor men's graphic t-shirt from Funny Guy Clothing! This shirt is perfect for anyone who likes to make others laugh, and it's sure to start a conversation wherever you go. The shirt is made from high-quality materials, and it's comfortable to wear. It's also machine-washable, so you can wear it again and again.

This shirt comes in a variety of colors, and it's available in sizes small to XXL. So whether you're shopping for yourself or someone else, this shirt is sure to put a smile on 

#### Be specific in your prompt
You can see that the model generated several versions. There may be times when you are looking for a stricter format - such as a single paragraph. Just ask the model to stick to it:

In [18]:
instruction = """Write an engaging product description for clothing eCommerce site. Make it a single paragraph.
Product: Humor Men's Graphic T-Shirt.
Description:

"""

response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.5,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This funny men's graphic t-shirt is perfect for anyone who enjoys a good joke! The design features a funny saying about beer, and it's sure to get a few laughs. The shirt is made of 100% cotton and is soft and comfortable. It's machine-washable and dryer-safe, and it comes in a variety of sizes.
Looking for the perfect t-shirt to wear to your next party? Look no further! This funny t-shirt is just what you need. It's made of high-quality materials and it's sure to make you stand out. Plus, it's super comfortable to wear.
This funny t-shirt is perfect for any guy who enjoys making people laugh! The graphic features an illustration of a man wearing sunglasses and holding up a sign that says "Yes, I'm The Fun One". It's sure to get a few laughs from your friends!


#### Personalize your prompt
The model can generate nice descriptions based on only an instruction, but they may not be aligned with your actual product. If you have special features, ask the model to include them:

In [20]:
instruction = """Write an engaging product description for clothing eCommerce site. Make sure to include the following features in the description.
Product: Humor Men's Graphic T-Shirt with a print of Einstein's quote: "artificial intelligence is no match for natural stupidity”
Features:
- Soft cotton
- Short sleeve
Description:

"""

response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This funny t-shirt is perfect for anyone who enjoys a good sense of humor. The print is of Albert Einstein's quote, "artificial intelligence is no match for natural stupidity." It's sure to get a few laughs, and it's a great way to show off your personality. The t-shirt is made of soft cotton, and it's comfortable to wear. It's available in a variety of sizes, so you can find the perfect fit.
This funny t-shirt is perfect for anyone who wants to look smart and stylish. The print features Albert Einstein's famous quote about artificial intelligence, "artificial intelligence is no match for natural stupidity". The shirt is made from soft cotton and is comfortable to wear. It's also machine washable, so you can keep it looking its best.
This funny graphic t-shirt is perfect for anyone who wants to stand out from the crowd. It's made of soft cotton and it features a print of Albert Einstein's quote "artificial intelligence is no match for natural stupidity."

Whether you're running errands

#### More format restrictions?
If you have several examples of your optimal output, you can feed the model with a few-shot prompt that includes several examples. Prompts like this are very useful when you want to enforce a certain format that is non-trivial to pick up from just one example. These prompts work best with base models; try to use them with [Jurassic-2 Grande](https://github.com/AI21Labs/SageMaker/blob/main/J2_Grande_example_model_use.ipynb) instead.

### D. Create your own product description!

Want to build your own product description? Here's your chance!

In [21]:
# Functions to build the prompt
def list_features(features):
    return "\n".join(
        f"- {feat}" for feat in features
    )

def build_prompt(product, features):
    example = "Write product descriptions for fashion eCommerce site. Make sure to include the following features in the description.\n"
    example += f"Product: {product}\n"
    example += "Features:\n"
    example += list_features(features)
    example += "\nDescription:\n"
    
    return example

In [20]:
product = input("Product: (for example, \'Women's Boho Beach Dress\')")
features = input("Features, separated by commas with no spaces: (for example, \'midi dress,swing hem,special for summer\')")
your_prompt = build_prompt(product, features.split(","))

Product: (for example, 'Women's Boho Beach Dress')Women's Boho Beach Dress
Features, separated by commas with no spaces: (for example, 'midi dress,swing hem,special for summer')midi dress,swing hem,special for summer


In [21]:
response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=your_prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This boho beach dress is perfect for summer days! It's made of lightweight, flowy fabric and has a swing hem that moves with you. The midi length is perfect for warm weather, and the dress can easily be dressed up or down.
This midi dress is perfect for summer! It's made from lightweight cotton and comes in a variety of colors, so you can choose your favorite. The swing hem adds a touch of fun and style, and the dress is loose and comfortable.
This boho beach dress is the perfect choice for your next vacation! The midi length is flattering and swing hem adds a touch of movement. The fabric is perfect for summer, and you'll love the colorful print. Whether you're headed to the beach or just relaxing at home, this dress is a must-have.


### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [22]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [23]:
model.delete_model()